In [1]:
from ipyleaflet import Map, basemaps, DrawControl
map = Map(
    center = ( 40, -2 ), 
    zoom = 8,
    basemap = basemaps.OpenTopoMap
)

draw_control = DrawControl()

draw_control.circle = {
    "shapeOptions": {
        "fillColor": None,
        "color": "#efed69",
        "fillOpacity": 0.2
    }
}

coordinates = []

def store_point( self, action, geo_json):
    global coordinates
    print(geo_json)
    coordinates = geo_json['geometry']['coordinates']
    
draw_control.on_draw(store_point)

map.add_control(draw_control)

map

Map(center=[40, -2], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

{'type': 'Feature', 'properties': {'style': {'stroke': True, 'color': '#efed69', 'weight': 4, 'opacity': 0.5, 'fill': True, 'fillColor': None, 'fillOpacity': 0.2, 'clickable': True, 'radius': 22497.689822625925}}, 'geometry': {'type': 'Point', 'coordinates': [-3.550212, 40.659667]}}
{'type': 'Feature', 'properties': {'style': {'stroke': True, 'color': '#efed69', 'weight': 4, 'opacity': 0.5, 'fill': True, 'fillColor': None, 'fillOpacity': 0.2, 'clickable': True, 'radius': 2139.791852366491}}, 'geometry': {'type': 'Point', 'coordinates': [-3.960363, 40.775827]}}


In [2]:
from pyproj import Proj, transform

print(coordinates)
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:25830')
x, y = transform(inProj, outProj, coordinates[0], coordinates[1])

width = 2000

minx = x - width / 2
maxx = x + width / 2
miny = y - width / 2
maxy = y + width / 2

inProj = Proj(init='epsg:25830')
outProj = Proj(init='epsg:4258')

a, b = transform(inProj, outProj, minx, miny)
c, d = transform(inProj, outProj, maxx, maxy)

bbox = (a, b, c, d)
print(bbox)

[-3.960363, 40.775827]
(-3.9720803438876744, 40.76672048172069, -3.948642456445934, 40.78493234977533)


In [14]:
from owslib.wcs import WebCoverageService
wcs = WebCoverageService('https://servicios.idee.es/wcs-inspire/mdt',version='1.0.0')

crs = wcs['Elevacion25830_5'].boundingboxes[0]['nativeSrs']
list(wcs.contents)
display(wcs['Elevacion4258_5'].supportedFormats)
# content = wcs.getCoverage(identifier='Elevacion25830_5', bbox=bbox, width=400, height=400, crs=crs, format='GEOTIFFINT16')
# out = open('prueba.tif','wb')

content = wcs.getCoverage(identifier='Elevacion25830_5', bbox=bbox, width=400, height=400, crs=crs, format='application/asc')

out = open('../src/images/mdt.asc', 'wb')
out.write(content.read())
out.close

['GEOTIFFINT16', 'ArcGrid', 'png', 'GEOTIFF_RGB']

<function BufferedWriter.close>

In [15]:

from owslib.wms import WebMapService
wms = WebMapService('https://www.ign.es/wms-inspire/mapa-raster', version='1.1.1')
list(wms.contents)
wms.getOperationByName('GetMap').formatOptions
img = wms.getmap( layers=['mtn_rasterizado'],
                srs='EPSG:25830',
                bbox=(minx, miny, maxx, maxy),
                format='image/png',
                size=(1024,1024)
                )
out = open('../src/images/topo.png', 'wb')
out.write(img.read())
out.close()

In [10]:
from owslib.wms import WebMapService
wms = WebMapService('https://www.ign.es/wms-inspire/pnoa-ma', version='1.1.1')
list(wms.contents)
wms.getOperationByName('GetMap').formatOptions
img = wms.getmap( layers=['OI.OrthoimageCoverage'],
                srs='EPSG:25830',
                bbox=(minx, miny, maxx, maxy),
                format='image/png',
                size=(1024,1024)
                )
out = open('pnoa.png', 'wb')
out.write(img.read())
out.close()

In [5]:
arguments = 'run -it -v /home/nano/firespread/src/images/:/home/nano usdaarsnwrc/katana'
!docker {arguments}

]0;root@ef59f7124d18: /root@ef59f7124d18:/# ^C

]0;root@ef59f7124d18: /root@ef59f7124d18:/# 